<a href="https://colab.research.google.com/github/TM0428/Brainfuck_editor/blob/master/system_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U beautifulsoup4

Requirement already up-to-date: beautifulsoup4 in /usr/local/lib/python3.6/dist-packages (4.9.3)


In [ ]:
# get html
import requests
import time
from pathlib import Path

def dirsName(name: str):
    return f"data/{name}/"

def fileName(name: str, id: str, extension: str="html")->str:
    return dirsName(name) + id + "." + extension

def urlName(article_url: str, article_id: str)->str:
    return article_url + article_id

def main():
    contents = [ # name, domain, article_urlは手動でせざるを得ないか、article_idを自動で取ってこられるようにしたい
        {
            "name": "sejuku",
            "domain":"https://www.sejuku.net/",
            "article_url":"https://www.sejuku.net/blog/",
            "article_id": ['33294', '3766', '105909'] # 記事idは https://www.sejuku.net/blog/curriculums-python から適当にとってきた。実際は https://www.sejuku.net/blog/archive/page/* から適当に取ってくるといいかもしれない
        },
        {
            "name": "techacademy",
            "domain":"https://techacademy.jp/",
            "article_url":"https://techacademy.jp/magazine/",
            "article_id": ['15508', '15636', '15571'] # 記事idは https://note.nkmk.me/ の新着記事・更新記事から適当にとってきた
        },
        {
            "name": "note_nkmk",
            "domain":"https://note.nkmk.me/",
            "article_url":"https://note.nkmk.me/",
            "article_id": ['python-union-find', 'python-numpy-sin-con-tan', 'python-list-2d-sort'] # 記事idは https://techacademy.jp/magazine/category/programming/python/page/44 から適当にとってきた
        },
        # https://docs.python.org/ はスクレイピングだめらしい
        
        #{
        #    "name": "python_docs",
        #    "domain":"https://docs.python.org/",
        #    "article_url":"https://docs.python.org/ja/3/library",
        #    "article_id": ['pathlib', 'random', 're'] # 記事idは https://docs.python.org/ja/3/library/ から適当にとってきた
        #},
        
        
    ]
    res = {}
    for content in contents:
        print(content)
        res[content['domain']] = []
        for id in content["article_id"]:
            filename = fileName(content["name"], id) # ローカル保存用のファイル名
            print(filename)
            file_path = Path(filename)
            print(file_path.exists())
            html_code:str
            if not file_path.exists(): # 1回getしたらローカルに保存して、複数回getしない
                urlname = urlName(content['article_url'], id) # get用のurl名
                print(urlname)
                html = requests.get(urlname) # getしたhtml(bytes)
                html_code = html.content.decode('utf-8') #getしたhtml(string)
                dir_path = Path(dirsName(content['name']))
                if not dir_path.exists():
                    dir_path.mkdir(mode=0o777,parents=True,exist_ok=False) # ディレクトリを作成
                with open(filename, mode='w', encoding='utf-8') as f:
                    f.write(html_code) # ローカルに保存
                time.sleep(3) # 適度なインターバル
            with open(filename, mode='r', encoding='utf-8') as f:
                html_code = f.read()
                res[content['domain']].append(html_code)
            
    return res




if __name__=="__main__":
    a = main()
    print(len(a['https://www.sejuku.net/'][0]))
    print(len(a['https://techacademy.jp/'][1]))
    print(len(a['https://note.nkmk.me/'][0]))



In [ ]:
# html to text
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import re

# text_setを生成する
# text_setは各葉ノードのテキストと木構造を表す
def soup2textset(tag, text_set, tag_string):
    for content in tag.contents:
        if isinstance(content, NavigableString):
            #content.string = content.string.replace('\xa9', '(c)').replace('\xa0', '')
            text_set.add(tag_string + content.string)
        elif str.lower(content.name) in ["script", "meta", "style", "header", "footer"]:
            pass
        else:
            #print(content.name)
            soup2textset(content, text_set, tag_string + str.lower(content.name))
    return text_set

# text_setに一致する葉ノードを削除する
def removetags(tag, remove_text_set, tag_string):
    remove_lists = []
    for content in tag.contents:
        if isinstance(content, NavigableString):
            if (tag_string+content.string) in remove_text_set:
                remove_lists.append(content)
        elif str.lower(content.name) in ["script", "meta", "style", "header", "footer"]:
            remove_lists.append(content)
        else:
            removetags(content, remove_text_set, tag_string + str.lower(content.name))
    for content in remove_lists:
        content.extract()

from typing import List
# Webページを取得して解析する
# def html2text(htmls: list(str)) -> list(str):
# 同じドメインのhtml(string)群を受け取って、タグを削除したstringのlistを返す
def html2text(htmls: List[str]) -> List[str]:
    # 文字列抽出
    soups = [BeautifulSoup(html.replace("</li>", "\n</li>"), "html.parser") for html in htmls]
    #bodies = [soup.find("body").text for soup in soups]

    bodies = [soup.find("body") for soup in soups]

    # 重複の削除
    if 2 <= len(bodies):
        # 重複要素の列挙
        set0 = soup2textset(bodies[0], set(), "")
        set1 = soup2textset(bodies[1], set(), "")
        remove_set = frozenset(set0 & set1)
        if 3 <= len(bodies):
            set2 = soup2textset(bodies[2], set(), "")
            remove_set = frozenset(remove_set | (set2 & set0)  | (set2 & set1))
        #print(remove_set)
        # 重複要素の削除
        for i in range(len(bodies)):
            removetags(bodies[i], remove_set, "")

    #soup => textの変換
    for i in range(len(bodies)):
        bodies[i] = bodies[i].text

    # 空行削除
    emptyline_regex = re.compile('\n\\s*\n')
    for i in range(len(bodies)):
        bodies[i] = emptyline_regex.sub('\n', bodies[i])

    return bodies

def test():
    import os
    html_names = os.listdir("./data/sejuku")
    htmls = [open("./data/sejuku/" + html, 'r', encoding="utf-8").read() for html in html_names]
    bodies = html2text(htmls)

    print(html_names[0])
    print(bodies[0][:100])
    print("........")
    print(bodies[0][-100:])
    print("====================")
    print(html_names[1])
    print(bodies[1][:100])
    print("........")
    print(bodies[1][-100:])
    print("====================")
    print(html_names[2])
    print(bodies[2][:100])
    print("........")
    print(bodies[2][-100:])
    print("====================")
    # print(bodies[0].replace('\xa9', '(c)').replace('\xa0', ''))

if __name__=="__main__":
    test()



33294.html

Pythonを自分のPCにインストールしたいけどどうやったらいいの？
Pythonのバージョンを確認するには？と疑問を持たれている方もいるのではないでしょうか？言語を本格的に学ぶためには環境を構築す
........
、Windows 10（64bit版）のインストール方法を解説します。（1）以下の公式サイトからインストーラーをダウンロードします。引用元：Python 　https:/www.python.org/
105909.html

Pythonの動かし方がいまいちわからない……
実行したらエラーが発生したけど、どうすればいいの？
おすすめの実行環境があったら教えてほしいこれからPythonを学習しようとしている初心者の方にとっ
........
おすすめの実行環境
Pythonのプログラムを実行する方法はそれほど難しくないと感じていただけたと思います。Pythonの実行方法はここで紹介した方法が基本となりますので、忘れないようにしましょう。 
3766.html
Cloud9を導入すれば、簡単にPythonの開発環境を構築できます！Python学習したい！そう思ってもプログラミングを始めるところにすらたどり着けないという方もいらっしゃるかと思います。今回はCl
........
見しました。
侍エンジニア塾は上記3つの成功ポイントを満たすようなサービス設計に磨きをかけております。
「自分のスタイルや目的に合わせて学習を進めたいな」とお考えの方は、ぜひチェックしてみてください。


In [ ]:
!pip install - -upgrade pip
!python -m pip install mecab-python3


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -u


In [ ]:
import MeCab
from gensim.models.doc2vec import Doc2Vec


class Embedder:
    def __init__(self, model_path="jawiki.doc2vec.dbow300d.model"):
        """
        constructor

        Args:
            model_path (str, optional): Defaults to "jawiki.doc2vec.dbow300d.model".
        """
        self.model = Doc2Vec.load(model_path)

    def _tokenize(self, text):
        """
        分かち書きを行うメソッド

        Args:
            text (str): 入力文字列

        Returns:
            list[str]: 分かち書き済み単語リスト
        """
        wakati = MeCab.Tagger("-O wakati")
        wakati.parse("")
        return wakati.parse(text).strip().split()

    def doc2vec(self, text):
        """
        doc2vec

        Args:
            text (str): string

        Returns:
            numpy.array: embedded vector
        """
        return self.model.infer_vector(self._tokenize(text))


def main():
    text = 'はじめに今まで自分が使ってきた中で、これは生産性が爆上げする！と思うものを厳選しました是非最後までご覧ください'
    embedder = Embedder()
    print(embedder.doc2vec(text))


if __name__ == "__main__":
  # データ一覧
  html_names = os.listdir("./data/sejuku")
  htmls = [open("./data/sejuku/" + html, 'r', encoding="utf-8").read() for html in html_names]
  bodies = html2text(htmls)
  embedder = Embedder()
  for body in bodies:
    print(embedder.doc2vec(body))

FileNotFoundError: ignored